<a href="https://colab.research.google.com/github/wlrma0108/Pytorch/blob/main/Bidirectional%20Recurrent%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.nn.utils import clip_grad_norm

In [4]:
device=torch.device=('cuda' if torch.cuda.is_available() else 'cpu')

sequence_length=28
input_size=28
hidden_size=128
num_layers=2
num_classes=10
batch_size=100
num_epochs=2
learning_rate=0.003

In [8]:
train_dataset=torchvision.datasets.MNIST(root='../../data/',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='../../data/',train=False,transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)


In [12]:
class BiRNN(nn.Module):
  def __init__(self,input_size,hidden_size, num_layers,num_classes):
    super(BiRNN,self).__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,bidirectional=True)
    self.fc=nn.Linear(hidden_size*2,num_classes)

  def forward(self,x):
    h0=torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)
    c0=torch.zeros(self.num_layers*2,x.size(0),self.hidden_size).to(device)

    out,_=self.lstm(x,(h0,c0))

    out=self.fc(out[:,-1,:])
    return out
model=BiRNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [13]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step=len(train_loader)
for epoch in range(num_epochs):
  for i, (images,labels)in enumerate(train_loader):
    images=images.reshape(-1,sequence_length,input_size).to(device)
    labels=labels.to(device)

    outputs=model(images)
    loss=criterion(outputs,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1)%100==0:
      print('Epoch[{}/{}], step []{}/{}],Loss:{:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item()))


with torch.no_grad():
  correct=0
  total=0
  for images,labels in test_loader:
    images=images.reshape(-1,sequence_length,input_size).to(device)
    labels=labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
  print('Test Accuracy og the model on the 10000 test images: {}%'.format(100*correct/total))
torch.save(model.state_dict(),'codel.ckpt')



Epoch[1/2], step []100/600],Loss:0.8190
Epoch[1/2], step []200/600],Loss:0.3203
Epoch[1/2], step []300/600],Loss:0.1788
Epoch[1/2], step []400/600],Loss:0.1465
Epoch[1/2], step []500/600],Loss:0.0848
Epoch[1/2], step []600/600],Loss:0.1022
Epoch[2/2], step []100/600],Loss:0.1851
Epoch[2/2], step []200/600],Loss:0.1549
Epoch[2/2], step []300/600],Loss:0.0856
Epoch[2/2], step []400/600],Loss:0.1152
Epoch[2/2], step []500/600],Loss:0.1085
Epoch[2/2], step []600/600],Loss:0.0567
Test Accuracy og the model on the 10000 test images: 97.44%
